In [54]:
data = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,20~1,1,21
"""
data = open('puzzle.data').read()



In [55]:
class Cube:
    def __init__(self, x, y, z, brick):
        self.x = x
        self.y = y
        self.z = z
        self.brick = brick
    
    def can_fall(self, cubes):
        return self.z != 1 and cubes.get((self.x, self.y, self.z-1), self.brick) == self.brick

class Brick:
    def __init__(self, p1, p2):
        if p1[0] != p2[0]:
            self.cubes = [Cube(x, p1[1], p1[2], self) for x in range(p1[0], p2[0]+1)]
        elif p1[1] != p2[1]:
            self.cubes = [Cube(p1[0], y, p1[2], self) for y in range(p1[1], p2[1]+1)]
        else:
            self.cubes = [Cube(p1[0], p1[1], z, self) for z in range(p1[2], p2[2]+1)]
    
    def can_fall(self, cubes):
        return all(cube.can_fall(cubes) for cube in self.cubes)



lines = [tuple(map(int, line.replace('~', ',').split(','))) for line in data.splitlines()]
lines.sort(key=lambda line: min(line[2], line[5]))
bricks = [Brick(line[0:3], line[3:6]) for line in lines]
cubes = {(cube.x, cube.y, cube.z): brick for brick in bricks for cube in brick.cubes}
    
def fall(bricks, cubes):
    falling_bricks = set()
    for brick in bricks:
        while brick.can_fall(cubes):
            for cube in brick.cubes:
                cubes[(cube.x, cube.y, cube.z - 1)] = cubes.pop( (cube.x, cube.y, cube.z) )
                cube.z -= 1
            falling_bricks.add(brick)
    return bricks, cubes, falling_bricks

bricks, cubes, _ = fall(bricks, cubes)

restore_cubes = {cube: (cube.x, cube.y, cube.z) for brick in bricks for cube in brick.cubes}
drop_map = {}

for brick in bricks:
    test_bricks = [b for b in bricks if b != brick]
    for cube, pos in restore_cubes.items():
        cube.x, cube.y, cube.z = pos
    test_cubes = {k: v for k, v in cubes.items() if v != brick}
    drop_map[brick] = fall(test_bricks, test_cubes)[2]

print(len([1 for v in drop_map.values() if len(v) == 0]))
print(sum(len(v) for v in drop_map.values()))



524
77070
